####  Adding features incrementally to observe the impact on accuracy to determine at which point the accuracy stops improving. This approach is commonly known as feature selection or feature engineering.

#### Adding features one by one and monitoring the accuracy helps in assessing the contribution of each feature to the predictive power of your model. This process helps identify the most informative features and can prevent overfitting by avoiding the inclusion of irrelevant or redundant features.

In [2]:
#Import all the necessary libraries
import pandas as pd
import pandas as pd
import os 
import numpy as nm
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential #importing Sequential Model.It uses tensorflow backend and you need to have tensorflow setup. 
from keras.layers import Dense #Importing default dense layer
from sklearn.utils import shuffle
from numpy.random import seed
from sklearn.metrics import accuracy_score
seed(1)

path_dir='C:/Users/subhi/AMNS_Second_phase_ANN_model/Datasets/'
filename1='Common_Data.xlsx'
filename2='Intersection_Data.xlsx'
#Access all the necessary variables and datasets
%store -r common_data 
%store -r random_forest_data
random_forest_data #161 statistical features and one output column
# Using the common_data dataframe created in 1_ANN_Model_on_common_features
#Usibg the appended_data dataframe created in Random_Forest_Code

,SPEEDMEAN,STOPPERMEAN,TUNDISHWEIGHTMEAN,TUNDISHTEMPERATUREMEAN,TUNDISHTEMPERATUREMIN,TUNDISHTEMPERATUREMAX,BROADFACE1WATERFLOWMEAN,BROADFACE1WATERFLOWVARIANCE,BROADFACE2WATERFLOWMEAN,BROADFACE2WATERFLOWVARIANCE,...,PER_ALCMEAN,PER_ALNMEAN,PER_NBCMEAN,PER_VNMEAN,PER_TINMEAN,PER_TICMEAN,PER_MNSMEAN,PER_VCMEAN,PER_NBNMEAN,OUTPUT
0,1.120270,50.758621,23.854885,1554.574713,1554.0,1555.0,3826.942529,5.341525,3820.425287,6.186947,...,0.002079,0.000137,0.001267,0.000024,0.000043,0.000651,0.002926,0.000371,0.000083,1
1,1.114253,56.146465,23.548081,1548.237374,1548.0,1549.0,3823.025253,9.519564,3823.176768,11.448551,...,0.002678,0.000180,0.001382,0.000026,0.000053,0.000788,0.003059,0.000387,0.000093,1
2,1.198736,61.086207,24.320115,1551.000000,1551.0,1551.0,3821.913793,6.366132,3821.626437,10.486887,...,0.002037,0.000163,0.001260,0.000028,0.000048,0.000595,0.003990,0.000350,0.000101,0
3,1.198899,65.455696,24.275696,1547.107595,1546.0,1548.0,3826.860759,8.828713,3826.151899,6.850344,...,0.001939,0.000144,0.001344,0.000026,0.000048,0.000651,0.002660,0.000350,0.000100,0
4,1.141729,66.005650,24.563785,1553.090395,1553.0,1554.0,3823.536723,9.514188,3823.525424,10.430145,...,0.002113,0.000151,0.001285,0.000027,0.000043,0.000607,0.003725,0.000378,0.000092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,0.983690,71.732143,23.541369,1545.142857,1545.0,1546.0,4000.803571,6.741178,4000.636905,9.136019,...,0.006482,0.000170,0.000421,0.000013,0.000010,0.000366,0.003682,0.000513,0.000011,0
587,0.795487,69.920635,27.779471,1555.682540,1554.0,1558.0,3958.724868,15125.331710,3966.460317,15426.523558,...,0.006992,0.000253,0.000426,0.000239,0.000004,0.000122,0.004050,0.006597,0.000015,1
588,0.911294,73.304124,22.814948,1537.123711,1536.0,1538.0,4052.396907,28.796073,4024.128866,11.359682,...,0.009100,0.000260,0.005642,0.000026,0.000081,0.002839,0.018383,0.000910,0.000161,1
589,0.918199,67.461165,24.742087,1543.466019,1542.0,1545.0,4030.233010,223.101046,4027.310680,8.719012,...,0.008918,0.000255,0.005460,0.000025,0.000076,0.002657,0.016292,0.000874,0.000156,1


### Training the base model 

In [2]:
common_data = shuffle(common_data, random_state=42)
#Defining train and test sizes
total_length= common_data.shape[0]
train_size=int(common_data.shape[0]*0.8)
test_size=common_data.shape[0]-train_size

# First converting dataframe series into numpy array and setting the datatype as float32 for all data
# In second step converting all 1D array into 2D array
train_X=common_data.iloc[:,:-1][0:train_size].to_numpy().astype(nm.float32) #2D array

train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array

test_X=common_data.iloc[:,:-1][train_size:].to_numpy().astype(nm.float32) #2D array

test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array

print('Dimension of training data',train_X.shape)
print('Dimension of testing data',test_X.shape)

#print(train_X)
print(type(train_X))
print(train_Y.shape)
test_Y.shape #2-D array
#train_Y

Dimension of training data (472, 8)
Dimension of testing data (119, 8)
<class 'numpy.ndarray'>
(472, 1)


(119, 1)

In [3]:
scaler=MinMaxScaler(feature_range=(0,1)) # scaler object

# First fit the scaler with your training data.
# It will find those parameters using which it is going to scale.
# Then transform any data you want.

train_X=scaler.fit_transform(train_X) 

# Since it has scaled  the data as per the parameters it learnt during training data.
# So it will require 2D array/similiar dimension as of training data.
# So testing data should be a 2D array.
# Testing data only need to be transformed on the basis of the parameters learnt using fit function on training data.

test_X=scaler.transform(test_X) #Scaling the training data and using the same parameters for testing data
#test
#train_X[:]

In [4]:
from keras.regularizers import l2
#Creating a Model
print('Total No of features',len(train_X[0,:])) #No of features
base_model=Sequential() 

base_model.add(Dense(50,input_dim=len(train_X[0,:]),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
base_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
base_model.summary()

Total No of features 8
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                450       
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________


In [5]:
#model.compile(optimizer='_' , loss='_' , metrics=['accuracy'])
base_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [6]:
#Fitting the training data
#base_model.fit(training data input, training data output, batch_size=_,validation_data=(x_test,y_test))
base_model.fit(train_X,train_Y,epochs=150,batch_size=5,validation_data=(test_X,test_Y))

# Epochs is number of times i want to do forward and backward propogation.
# batch size is used for update the weights in mini batches. The default back size is 32.
# Validation data : accuracy looking like on validation data after every epoch, you can give test data here too. 
#The loss will be reduced due to backpropgation through time.

#Add regularization or increase number of epochs for better accuracy.

Epoch 1/150
95/95 [==============================] - 1s 4ms/step - loss: 0.8244 - accuracy: 0.4555 - val_loss: 0.7960 - val_accuracy: 0.4622
Epoch 2/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7728 - accuracy: 0.5148 - val_loss: 0.7596 - val_accuracy: 0.5294
Epoch 3/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7467 - accuracy: 0.5530 - val_loss: 0.7403 - val_accuracy: 0.5210
Epoch 4/150
95/95 [==============================] - 0s 3ms/step - loss: 0.7329 - accuracy: 0.5254 - val_loss: 0.7323 - val_accuracy: 0.4874
Epoch 5/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7227 - accuracy: 0.5784 - val_loss: 0.7216 - val_accuracy: 0.5714
Epoch 6/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7163 - accuracy: 0.5975 - val_loss: 0.7172 - val_accuracy: 0.6134
Epoch 7/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7113 - accuracy: 0.5953 - val_loss: 0.7186 - val_accuracy: 0.5042
Epoch 8/150
9

Epoch 59/150
95/95 [==============================] - 0s 3ms/step - loss: 0.6909 - accuracy: 0.6377 - val_loss: 0.6969 - val_accuracy: 0.6218
Epoch 60/150
95/95 [==============================] - 0s 3ms/step - loss: 0.6903 - accuracy: 0.6250 - val_loss: 0.6988 - val_accuracy: 0.5966
Epoch 61/150
95/95 [==============================] - 0s 3ms/step - loss: 0.6898 - accuracy: 0.6377 - val_loss: 0.6998 - val_accuracy: 0.5630
Epoch 62/150
95/95 [==============================] - 0s 3ms/step - loss: 0.6916 - accuracy: 0.6419 - val_loss: 0.7004 - val_accuracy: 0.5462
Epoch 63/150
95/95 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.6314 - val_loss: 0.7044 - val_accuracy: 0.5630
Epoch 64/150
95/95 [==============================] - 0s 3ms/step - loss: 0.6898 - accuracy: 0.6229 - val_loss: 0.6990 - val_accuracy: 0.5882
Epoch 65/150
95/95 [==============================] - 0s 3ms/step - loss: 0.6903 - accuracy: 0.6398 - val_loss: 0.7013 - val_accuracy: 0.5378
Epoch 

In [21]:
# Create a list to store the selected features and their corresponding accuracies
base_features = common_data.columns.tolist()[:-1]
base_features

['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'STEELLEVELINMOLD-SETMEAN',
 'TUNDISHWEIGHTMEAN',
 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'PER_NIMEAN',
 'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDTAPERPERCENTMEAN']

In [22]:
# Evaluate the model

testPredict_probs=base_model.predict(test_X)
#testPredict_probs

# Apply threshold to obtain predicted classes
threshold = 0.5  # Adjust this threshold as needed
y_pred = (testPredict_probs > threshold).astype(int)
y_pred

base_accuracy = accuracy_score(test_Y,y_pred)
base_accuracy

4/4 [==============================] - 0s 3ms/step


0.5714285714285714

### Data for new model

In [23]:
%store -r feature_imp_RF
total_features=feature_imp_RF["Feature"]
total_features_list= total_features.tolist()

remaining_features=[]
for feature in total_features_list:
    if feature not in base_features:
        remaining_features.append(feature) 
        
len(remaining_features)

153

### Iterating over remaining features

In [25]:
new_features=[]
count=0
optimal_features=[]
accuracy_store=[base_accuracy]

# Start the timer
start_time = time.time()

for feature in remaining_features:
    # Add one feature to the feature set
    new_features = base_features + [feature]
    
    #Creating a new data
    new_data=random_forest_data.loc[:,new_features]
    new_data = shuffle(new_data, random_state=42)
    
    #Splitting the new_data into train and test
    new_total_length= new_data.shape[0]
    new_train_size=int(new_data.shape[0]*0.8)
    new_test_size=new_data.shape[0]-train_size
    new_train_X=new_data.iloc[:,:][0:train_size].to_numpy().astype(nm.float32) #2D array
    new_train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
    new_train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array
    new_test_X=new_data.iloc[:,:][train_size:].to_numpy().astype(nm.float32) #2D array
    new_test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
    new_test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array}

    
    # Create a new model with the updated feature set
    new_model=Sequential() 
    new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
    new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
    # Compile the new model
    new_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    print(count)
    print(feature)
    
    
    # Train the new model
    new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=5,validation_data=(new_test_X,new_test_Y))
    
    # Evaluate the new model
    testPredict_probs_new=new_model.predict(new_test_X)
    threshold = 0.5  # Adjust this threshold as needed
    y_new_pred = (testPredict_probs_new > threshold).astype(int)
    new_accuracy = accuracy_score(new_test_Y,y_new_pred)
    accuracy_store.append(new_accuracy)
    
    # Update the baseline feature set if the accuracy score improved
    if new_accuracy> base_accuracy:
        base_features = new_features
        base_accuracy = new_accuracy
        optimal_features.append(new_features)
    
    # Check for convergence
    #if len(base_features) == random_forest_data.iloc[:,:-1].shape[1]:
     #   break
        
# Stop the timer and calculate the elapsed time
end_time = time.time()
training_time = end_time - start_time

0
MOLDWIDTHTOPMEAN
Epoch 1/150
95/95 [==============================] - 1s 5ms/step - loss: 4.7609 - accuracy: 0.4852 - val_loss: 1.0954 - val_accuracy: 0.4286
Epoch 2/150
95/95 [==============================] - 0s 3ms/step - loss: 1.5643 - accuracy: 0.4746 - val_loss: 0.9518 - val_accuracy: 0.5126
Epoch 3/150
95/95 [==============================] - 0s 4ms/step - loss: 1.4401 - accuracy: 0.5169 - val_loss: 1.0116 - val_accuracy: 0.5630
Epoch 4/150
95/95 [==============================] - 0s 3ms/step - loss: 1.2411 - accuracy: 0.5042 - val_loss: 0.9777 - val_accuracy: 0.5630
Epoch 5/150
95/95 [==============================] - 0s 3ms/step - loss: 2.0833 - accuracy: 0.5085 - val_loss: 3.1867 - val_accuracy: 0.4370
Epoch 6/150
95/95 [==============================] - 0s 3ms/step - loss: 1.3105 - accuracy: 0.5106 - val_loss: 0.8655 - val_accuracy: 0.4118
Epoch 7/150
95/95 [==============================] - 0s 4ms/step - loss: 1.7493 - accuracy: 0.4894 - val_loss: 1.8051 - val_accuracy: 0

Epoch 116/150
95/95 [==============================] - 0s 4ms/step - loss: 1.3964 - accuracy: 0.5212 - val_loss: 0.7128 - val_accuracy: 0.5378
Epoch 117/150
95/95 [==============================] - 0s 4ms/step - loss: 1.2429 - accuracy: 0.5042 - val_loss: 0.7228 - val_accuracy: 0.5966
Epoch 118/150
95/95 [==============================] - 0s 4ms/step - loss: 1.3619 - accuracy: 0.5106 - val_loss: 2.2734 - val_accuracy: 0.5630
Epoch 119/150
95/95 [==============================] - 0s 4ms/step - loss: 1.3858 - accuracy: 0.5212 - val_loss: 1.9877 - val_accuracy: 0.4370
Epoch 120/150
95/95 [==============================] - 0s 4ms/step - loss: 1.0057 - accuracy: 0.5297 - val_loss: 1.7343 - val_accuracy: 0.4370
Epoch 121/150
95/95 [==============================] - 0s 4ms/step - loss: 1.0867 - accuracy: 0.5127 - val_loss: 2.0474 - val_accuracy: 0.5630
Epoch 122/150
95/95 [==============================] - 0s 4ms/step - loss: 1.1376 - accuracy: 0.5127 - val_loss: 0.7212 - val_accuracy: 0.5630

Epoch 23/150
95/95 [==============================] - 0s 3ms/step - loss: 1.4960 - accuracy: 0.5106 - val_loss: 0.8576 - val_accuracy: 0.5630
Epoch 24/150
95/95 [==============================] - 0s 4ms/step - loss: 1.1391 - accuracy: 0.5381 - val_loss: 0.9057 - val_accuracy: 0.5966
Epoch 25/150
95/95 [==============================] - 0s 4ms/step - loss: 1.3363 - accuracy: 0.5169 - val_loss: 1.1753 - val_accuracy: 0.5714
Epoch 26/150
95/95 [==============================] - 0s 5ms/step - loss: 2.0614 - accuracy: 0.4831 - val_loss: 1.0251 - val_accuracy: 0.5882
Epoch 27/150
95/95 [==============================] - 0s 4ms/step - loss: 1.4574 - accuracy: 0.5127 - val_loss: 0.9242 - val_accuracy: 0.5126
Epoch 28/150
95/95 [==============================] - 0s 3ms/step - loss: 1.6973 - accuracy: 0.5487 - val_loss: 2.2847 - val_accuracy: 0.5630
Epoch 29/150
95/95 [==============================] - 0s 3ms/step - loss: 1.2183 - accuracy: 0.4788 - val_loss: 1.1164 - val_accuracy: 0.4622
Epoch 

Epoch 81/150
95/95 [==============================] - 0s 4ms/step - loss: 1.2633 - accuracy: 0.5085 - val_loss: 0.7499 - val_accuracy: 0.5546
Epoch 82/150
95/95 [==============================] - 0s 4ms/step - loss: 0.9469 - accuracy: 0.5275 - val_loss: 1.9518 - val_accuracy: 0.4370
Epoch 83/150
95/95 [==============================] - 0s 4ms/step - loss: 2.1219 - accuracy: 0.5106 - val_loss: 0.9509 - val_accuracy: 0.5714
Epoch 84/150
95/95 [==============================] - 0s 3ms/step - loss: 1.3088 - accuracy: 0.5254 - val_loss: 0.7809 - val_accuracy: 0.5378
Epoch 85/150
95/95 [==============================] - 0s 4ms/step - loss: 1.7672 - accuracy: 0.5254 - val_loss: 0.7712 - val_accuracy: 0.5798
Epoch 86/150
95/95 [==============================] - 0s 3ms/step - loss: 1.2687 - accuracy: 0.5530 - val_loss: 0.7963 - val_accuracy: 0.5378
Epoch 87/150
95/95 [==============================] - 0s 3ms/step - loss: 1.2870 - accuracy: 0.5169 - val_loss: 1.2340 - val_accuracy: 0.4538
Epoch 

95/95 [==============================] - 0s 3ms/step - loss: 1.0266 - accuracy: 0.5403 - val_loss: 2.7634 - val_accuracy: 0.4454
Epoch 46/150
95/95 [==============================] - 0s 3ms/step - loss: 1.6575 - accuracy: 0.5275 - val_loss: 0.8217 - val_accuracy: 0.5630
Epoch 47/150
95/95 [==============================] - 0s 3ms/step - loss: 1.0685 - accuracy: 0.5381 - val_loss: 1.2775 - val_accuracy: 0.4790
Epoch 48/150
95/95 [==============================] - 0s 3ms/step - loss: 1.1827 - accuracy: 0.5403 - val_loss: 0.8887 - val_accuracy: 0.5546
Epoch 49/150
95/95 [==============================] - 0s 3ms/step - loss: 1.2113 - accuracy: 0.5636 - val_loss: 2.3665 - val_accuracy: 0.4454
Epoch 50/150
95/95 [==============================] - 0s 3ms/step - loss: 1.5470 - accuracy: 0.5636 - val_loss: 1.3111 - val_accuracy: 0.4790
Epoch 51/150
95/95 [==============================] - 0s 3ms/step - loss: 1.6249 - accuracy: 0.5275 - val_loss: 2.4572 - val_accuracy: 0.5630
Epoch 52/150
95/95 

...

Epoch 101/150
95/95 [==============================] - 0s 1ms/step - loss: 1.1105 - accuracy: 0.6123 - val_loss: 2.0446 - val_accuracy: 0.5630
Epoch 102/150
95/95 [==============================] - 0s 1ms/step - loss: 0.8743 - accuracy: 0.6250 - val_loss: 0.7201 - val_accuracy: 0.6218
Epoch 103/150
95/95 [==============================] - 0s 1ms/step - loss: 1.3641 - accuracy: 0.5911 - val_loss: 2.0711 - val_accuracy: 0.5630
Epoch 104/150
95/95 [==============================] - 0s 1ms/step - loss: 1.1949 - accuracy: 0.5847 - val_loss: 0.7640 - val_accuracy: 0.7059
Epoch 105/150
95/95 [==============================] - 0s 1ms/step - loss: 1.0716 - accuracy: 0.5826 - val_loss: 2.4969 - val_accuracy: 0.5630
Epoch 106/150
95/95 [==============================] - 0s 1ms/step - loss: 1.2220 - accuracy: 0.5869 - val_loss: 0.9732 - val_accuracy: 0.5966
Epoch 107/150
95/95 [==============================] - 0s 1ms/step - loss: 1.0729 - accuracy: 0.5869 - val_loss: 1.7821 - val_accuracy: 0.4706

Epoch 8/150
95/95 [==============================] - 0s 1ms/step - loss: 2.2620 - accuracy: 0.5318 - val_loss: 1.7850 - val_accuracy: 0.5630
Epoch 9/150
95/95 [==============================] - 0s 1ms/step - loss: 1.3175 - accuracy: 0.5169 - val_loss: 2.0488 - val_accuracy: 0.5630
Epoch 10/150
95/95 [==============================] - 0s 1ms/step - loss: 1.6664 - accuracy: 0.5403 - val_loss: 2.9961 - val_accuracy: 0.4370
Epoch 11/150
95/95 [==============================] - 0s 1ms/step - loss: 1.3881 - accuracy: 0.5445 - val_loss: 0.7485 - val_accuracy: 0.5546
Epoch 12/150
95/95 [==============================] - 0s 1ms/step - loss: 1.4462 - accuracy: 0.5381 - val_loss: 1.2912 - val_accuracy: 0.5630
Epoch 13/150
95/95 [==============================] - 0s 1ms/step - loss: 1.5036 - accuracy: 0.5233 - val_loss: 1.8447 - val_accuracy: 0.5630
Epoch 14/150
95/95 [==============================] - 0s 1ms/step - loss: 1.1178 - accuracy: 0.5572 - val_loss: 2.4628 - val_accuracy: 0.5630
Epoch 15

Epoch 66/150
95/95 [==============================] - 0s 1ms/step - loss: 0.9754 - accuracy: 0.5636 - val_loss: 0.6987 - val_accuracy: 0.6723
Epoch 67/150
95/95 [==============================] - 0s 1ms/step - loss: 0.9166 - accuracy: 0.5911 - val_loss: 0.7960 - val_accuracy: 0.6723
Epoch 68/150
95/95 [==============================] - 0s 1ms/step - loss: 1.4727 - accuracy: 0.5297 - val_loss: 0.7879 - val_accuracy: 0.6050
Epoch 69/150
95/95 [==============================] - 0s 1ms/step - loss: 1.1574 - accuracy: 0.5720 - val_loss: 0.8271 - val_accuracy: 0.6218
Epoch 70/150
95/95 [==============================] - 0s 1ms/step - loss: 1.0586 - accuracy: 0.5784 - val_loss: 1.7920 - val_accuracy: 0.4454
Epoch 71/150
95/95 [==============================] - 0s 1ms/step - loss: 0.9729 - accuracy: 0.5890 - val_loss: 2.0495 - val_accuracy: 0.4538
Epoch 72/150
95/95 [==============================] - 0s 1ms/step - loss: 1.0297 - accuracy: 0.5826 - val_loss: 0.7095 - val_accuracy: 0.6050
Epoch 

95/95 [==============================] - 0s 1ms/step - loss: 0.8970 - accuracy: 0.6038 - val_loss: 1.9835 - val_accuracy: 0.4370
Epoch 31/150
95/95 [==============================] - 0s 1ms/step - loss: 1.1120 - accuracy: 0.5720 - val_loss: 1.0156 - val_accuracy: 0.5042
Epoch 32/150
95/95 [==============================] - 0s 1ms/step - loss: 1.2269 - accuracy: 0.5720 - val_loss: 2.2910 - val_accuracy: 0.5630
Epoch 33/150
95/95 [==============================] - 0s 1ms/step - loss: 1.1080 - accuracy: 0.5466 - val_loss: 2.2969 - val_accuracy: 0.4370
Epoch 34/150
95/95 [==============================] - 0s 1ms/step - loss: 1.4464 - accuracy: 0.5106 - val_loss: 1.4703 - val_accuracy: 0.4370
Epoch 35/150
95/95 [==============================] - 0s 1ms/step - loss: 1.2347 - accuracy: 0.5339 - val_loss: 0.7092 - val_accuracy: 0.6050
Epoch 36/150
95/95 [==============================] - 0s 1ms/step - loss: 1.0039 - accuracy: 0.5275 - val_loss: 0.7016 - val_accuracy: 0.6387
Epoch 37/150
95/95 

Epoch 88/150
95/95 [==============================] - 0s 1ms/step - loss: 2.5382 - accuracy: 0.5424 - val_loss: 1.0432 - val_accuracy: 0.5378
Epoch 89/150
95/95 [==============================] - 0s 1ms/step - loss: 1.0812 - accuracy: 0.5996 - val_loss: 0.6895 - val_accuracy: 0.6555
Epoch 90/150
95/95 [==============================] - 0s 1ms/step - loss: 1.5182 - accuracy: 0.5085 - val_loss: 0.9331 - val_accuracy: 0.6134
Epoch 91/150
95/95 [==============================] - 0s 1ms/step - loss: 0.8687 - accuracy: 0.6229 - val_loss: 0.7380 - val_accuracy: 0.6387
Epoch 92/150
95/95 [==============================] - 0s 1ms/step - loss: 0.8906 - accuracy: 0.5975 - val_loss: 1.4255 - val_accuracy: 0.5630
Epoch 93/150
95/95 [==============================] - 0s 1ms/step - loss: 1.3231 - accuracy: 0.5932 - val_loss: 1.1524 - val_accuracy: 0.5210
Epoch 94/150
95/95 [==============================] - 0s 1ms/step - loss: 1.1635 - accuracy: 0.6059 - val_loss: 0.7401 - val_accuracy: 0.6387
Epoch 

In [33]:
print("Simulation ended",training_time)
print("Best feature set:", base_features)
print("Best accuracy score:", base_accuracy)

Simulation ended 5447.2976360321045
Best feature set: ['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN', 'STEELLEVELINMOLD-SETMEAN', 'TUNDISHWEIGHTMEAN', 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'PER_NIMEAN', 'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'MOLDTAPERPERCENTMEAN', 'MOLDWIDTHTOPMEAN', 'SEG7+8BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'PER_MOMEAN', 'STRANDDRIVECURRENTMIN', 'SEG7+8BOTTOMSECONDARYCOOLINGWATERPRESSUREACTUALMEAN']
Best accuracy score: 0.6974789915966386


In [36]:
accuracy_store

[0.5714285714285714,
 0.5798319327731093,
 0.5630252100840336,
 0.48739495798319327,
 0.4369747899159664,
 0.5210084033613446,
 0.5882352941176471,
 0.453781512605042,
 0.6386554621848739,
 0.5630252100840336,
 0.5966386554621849,
 0.6134453781512605,
 0.5378151260504201,
 0.5798319327731093,
 0.6134453781512605,
 0.680672268907563,
 0.4957983193277311,
 0.5126050420168067,
 0.6134453781512605,
 0.6722689075630253,
 0.46218487394957986,
 0.5798319327731093,
 0.6050420168067226,
 0.5630252100840336,
 0.680672268907563,
 0.4369747899159664,
 0.6722689075630253,
 0.5630252100840336,
 0.6218487394957983,
 0.6218487394957983,
 0.6554621848739496,
 0.5126050420168067,
 0.6722689075630253,
 0.6386554621848739,
 0.6134453781512605,
 0.6134453781512605,
 0.4369747899159664,
 0.5630252100840336,
 0.46218487394957986,
 0.5798319327731093,
 0.5630252100840336,
 0.5882352941176471,
 0.453781512605042,
 0.680672268907563,
 0.6386554621848739,
 0.5798319327731093,
 0.5798319327731093,
 0.663865546218

In [38]:
%store accuracy_store
%store base_features
%store base_accuracy
%store optimal_features

Stored 'accuracy_store' (list)
Stored 'base_features' (list)
Stored 'base_accuracy' (float64)
Stored 'optimal_features' (list)


In [29]:
len(base_features)
base_features

['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'STEELLEVELINMOLD-SETMEAN',
 'TUNDISHWEIGHTMEAN',
 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'PER_NIMEAN',
 'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDTAPERPERCENTMEAN',
 'MOLDWIDTHTOPMEAN',
 'SEG7+8BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'PER_MOMEAN',
 'STRANDDRIVECURRENTMIN',
 'SEG7+8BOTTOMSECONDARYCOOLINGWATERPRESSUREACTUALMEAN']

In [34]:
common_data.columns

Index(['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
       'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
       'STEELLEVELINMOLD-SETMEAN', 'TUNDISHWEIGHTMEAN',
       'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'PER_NIMEAN',
       'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
       'MOLDTAPERPERCENTMEAN', 'OUTPUT'],
      dtype='object')

In [11]:
new_features=[]
for feature in remaining_features:
    # Add one feature to the feature set
    new_features = base_features + [feature]

In [12]:
new_features

['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'STEELLEVELINMOLD-SETMEAN',
 'TUNDISHWEIGHTMEAN',
 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'PER_NIMEAN',
 'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDTAPERPERCENTMEAN',
 'SPEEDMEAN']

In [13]:
#Creating a new data
new_data=random_forest_data.loc[:,new_features]
new_data = shuffle(new_data, random_state=42)
new_data.shape

(591, 9)

In [14]:
 #Splitting the new_data into train and test
new_total_length= new_data.shape[0]
new_train_size=int(new_data.shape[0]*0.8)
new_test_size=new_data.shape[0]-train_size
new_train_X=new_data.iloc[:,:][0:train_size].to_numpy().astype(nm.float32) #2D array
new_train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
new_train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array
new_test_X=new_data.iloc[:,:][train_size:].to_numpy().astype(nm.float32) #2D array
new_test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
new_test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array

In [15]:
new_train_X.shape

(472, 9)

In [16]:
# Create a new model with the updated feature set
new_model=Sequential() 
new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer

In [146]:
# Compile the new model
new_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
# Train the new model
new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=5,validation_data=(new_test_X,new_test_Y))

# Evaluate the new model
testPredict_probs_new=new_model.predict(new_test_X)
threshold = 0.5  # Adjust this threshold as needed
y_new_pred = (testPredict_probs_new > threshold).astype(int)
new_accuracy = accuracy_score(new_test_Y,y_new_pred)
    

Epoch 1/150
95/95 [==============================] - 1s 3ms/step - loss: 1.6032 - accuracy: 0.4767 - val_loss: 1.0520 - val_accuracy: 0.4874
Epoch 2/150
95/95 [==============================] - 0s 1ms/step - loss: 0.9668 - accuracy: 0.4534 - val_loss: 1.2166 - val_accuracy: 0.4286
Epoch 3/150
95/95 [==============================] - 0s 1ms/step - loss: 0.9210 - accuracy: 0.4661 - val_loss: 0.8984 - val_accuracy: 0.4622
Epoch 4/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8672 - accuracy: 0.4725 - val_loss: 0.8847 - val_accuracy: 0.5294
Epoch 5/150
95/95 [==============================] - 0s 1ms/step - loss: 0.8145 - accuracy: 0.5106 - val_loss: 0.9147 - val_accuracy: 0.4286
Epoch 6/150
95/95 [==============================] - 0s 1ms/step - loss: 0.8167 - accuracy: 0.4915 - val_loss: 0.8115 - val_accuracy: 0.4538
Epoch 7/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7997 - accuracy: 0.4640 - val_loss: 0.7822 - val_accuracy: 0.4538
Epoch 8/150
9

Epoch 59/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7291 - accuracy: 0.5869 - val_loss: 0.6906 - val_accuracy: 0.5630
Epoch 60/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7176 - accuracy: 0.5657 - val_loss: 0.6899 - val_accuracy: 0.5462
Epoch 61/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7220 - accuracy: 0.5699 - val_loss: 0.6933 - val_accuracy: 0.5798
Epoch 62/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7089 - accuracy: 0.5996 - val_loss: 0.7447 - val_accuracy: 0.5294
Epoch 63/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7585 - accuracy: 0.5805 - val_loss: 0.6937 - val_accuracy: 0.5798
Epoch 64/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7225 - accuracy: 0.5445 - val_loss: 0.6925 - val_accuracy: 0.5798
Epoch 65/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7063 - accuracy: 0.5890 - val_loss: 0.6884 - val_accuracy: 0.5462
Epoch 

In [158]:
random_forest_data.iloc[:,:-1].shape[1]


161

In [17]:
import time

# Compile the new model
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Start the timer
start_time = time.time()

# Train the new model
new_model.fit(new_train_X, new_train_Y, epochs=150, batch_size=5, validation_data=(new_test_X, new_test_Y))

# Stop the timer and calculate the elapsed time
end_time = time.time()
training_time = end_time - start_time

# Evaluate the new model
testPredict_probs_new = new_model.predict(new_test_X)
threshold = 0.5  # Adjust this threshold as needed
y_new_pred = (testPredict_probs_new > threshold).astype(int)
new_accuracy = accuracy_score(new_test_Y, y_new_pred)

# Print the training time
print("Training Time: {:.2f} seconds".format(training_time))


Epoch 1/150
95/95 [==============================] - 1s 4ms/step - loss: 0.9540 - accuracy: 0.5339 - val_loss: 0.9819 - val_accuracy: 0.4118
Epoch 2/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8895 - accuracy: 0.4788 - val_loss: 0.8797 - val_accuracy: 0.4118
Epoch 3/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8629 - accuracy: 0.4852 - val_loss: 0.8526 - val_accuracy: 0.4202
Epoch 4/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8230 - accuracy: 0.4979 - val_loss: 0.8514 - val_accuracy: 0.4118
Epoch 5/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8401 - accuracy: 0.4703 - val_loss: 0.7900 - val_accuracy: 0.4286
Epoch 6/150
95/95 [==============================] - 0s 3ms/step - loss: 0.7832 - accuracy: 0.5318 - val_loss: 0.8835 - val_accuracy: 0.4370
Epoch 7/150
95/95 [==============================] - 0s 3ms/step - loss: 0.8159 - accuracy: 0.4746 - val_loss: 0.7572 - val_accuracy: 0.5378
Epoch 8/150
9

Epoch 59/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7085 - accuracy: 0.5699 - val_loss: 0.7057 - val_accuracy: 0.5378
Epoch 60/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7106 - accuracy: 0.5763 - val_loss: 0.7137 - val_accuracy: 0.5882
Epoch 61/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7261 - accuracy: 0.5805 - val_loss: 0.7059 - val_accuracy: 0.5546
Epoch 62/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7011 - accuracy: 0.5720 - val_loss: 0.6995 - val_accuracy: 0.5546
Epoch 63/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7090 - accuracy: 0.5678 - val_loss: 0.6911 - val_accuracy: 0.5798
Epoch 64/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7293 - accuracy: 0.5297 - val_loss: 0.7111 - val_accuracy: 0.5378
Epoch 65/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7450 - accuracy: 0.5636 - val_loss: 0.7623 - val_accuracy: 0.5042
Epoch 

In [19]:
a=[1]
a.append(2)

In [20]:
a

[1, 2]

In [30]:
a=0.69
b=0.5

In [31]:
print(a>b)

True


In [32]:
print(b>a)

False
